In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [51]:
dfx=pd.read_csv('train.csv')
dfx=pd.DataFrame(dfx)
#print(dfx.head(n=10))
dfx=dfx.drop(['Name','PassengerId','Ticket','Cabin'],axis=1)

label=LabelEncoder()
dfx.Sex=label.fit_transform(dfx.Sex)
dfx.Embarked=dfx.Embarked.fillna('S')
z=dfx.Age.mean()
dfx.Age=dfx.Age.fillna(int(z))
dfx.Embarked=label.fit_transform(dfx.Embarked)
dfy=dfx.Survived
dfx=dfx.drop(['Survived'],axis=1)
print(dfx.info())
print(dfx.head())
X=dfx.values
Y=dfy.values
#print(X)
print(Y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null int32
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null int32
dtypes: float64(2), int32(2), int64(3)
memory usage: 41.9 KB
None
   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0       3    1  22.0      1      0   7.2500         2
1       1    0  38.0      1      0  71.2833         0
2       3    0  26.0      0      0   7.9250         2
3       1    0  35.0      1      0  53.1000         2
4       3    1  35.0      0      0   8.0500         2
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0

In [52]:
ques=pd.read_csv('test.csv')
ques=pd.DataFrame(ques)
ques=ques.drop(['Name','PassengerId','Ticket','Cabin'],axis=1)
ques.Sex=label.fit_transform(ques.Sex)
ques.Embarked=ques.Embarked.fillna('S')
z=ques.Age.mean()
ques.Age=ques.Age.fillna(int(z))
ques.Embarked=label.fit_transform(ques.Embarked)
print(ques.head(n=10))
z=ques.Fare.mean()
ques.Fare=ques.Fare.fillna(float(z))
ques.info()
x=ques.values
print(x[0])

   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0       3    1  34.5      0      0   7.8292         1
1       3    0  47.0      1      0   7.0000         2
2       2    1  62.0      0      0   9.6875         1
3       3    1  27.0      0      0   8.6625         2
4       3    0  22.0      1      1  12.2875         2
5       3    1  14.0      0      0   9.2250         2
6       3    0  30.0      0      0   7.6292         1
7       2    1  26.0      1      1  29.0000         2
8       3    0  18.0      0      0   7.2292         0
9       3    1  21.0      2      0  24.1500         2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null int32
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Embarked    418 non-null int32
dtypes: float64(2), int32(2), int64(3)
memory usage: 19.7 KB
[ 3.      1.     34.

In [53]:
def sigmoid(x):
    if x >= 0:
        return 1. / ( 1. + np.exp(-x) )
    else:
        return np.exp(x) / ( 1. + np.exp(x) )

def hypothesis(x,w,b):
    
    h = np.dot(x,w) + b
    return sigmoid(h)

def safeLog(x):

    xSafe = x + 1e-10
    return np.log(xSafe)

def error(y_true,x,w,b):
    
    m = x.shape[0]
    
    err = 0.0
    
    for i in range(m):
        hx = hypothesis(x[i],w,b) 
        
        err += y_true[i]*safeLog(hx) + (1-y_true[i])*safeLog(1-hx)
        
    
    return -err/m


def get_grads(y_true,x,w,b):
    
    grad_w = np.zeros(w.shape)
    grad_b = 0.0
    
    m = x.shape[0]
    
    for i in range(m):
        hx = hypothesis(x[i],w,b)
        
        grad_w += (y_true[i] - hx)*x[i]
        grad_b +=  (y_true[i]-hx)
        
    
    grad_w /= m
    grad_b /= m
    
    return [grad_w,grad_b]


# One Iteration of Gradient Descent
def grad_descent(x,y_true,w,b,learning_rate=0.1):
    
    err = error(y_true,x,w,b)
    [grad_w,grad_b] = get_grads(y_true,x,w,b)
    
    w = w + learning_rate*grad_w
    b = b + learning_rate*grad_b
    
    return err,w,b
    
def predict(x,w,b):
    
    confidence = hypothesis(x,w,b)
    if confidence<0.5:
        return 0
    else:
        return 1
    
def get_acc(x_tst,y_tst,w,b):
    
    y_pred = []
    
    for i in range(y_tst.shape[0]):
        p = predict(x_tst[i],w,b)
        y_pred.append(p)
        
    y_pred = np.array(y_pred)
    
    return  float((y_pred==y_tst).sum())/y_tst.shape[0]
        
    

In [54]:
loss = []
acc = []

W = 2*np.random.random((X.shape[1],))
b = 5*np.random.random()

In [55]:
test=pd.read_csv('gender_submission.csv')
test=pd.DataFrame(test)

print(test.head())
a=test.PassengerId
test=test.drop(['PassengerId'],axis=1)
print(test.head())
test=test.values
test=test.reshape(-1)
print(test)

   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1
   Survived
0         0
1         1
2         0
3         0
4         1
[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1

In [59]:
for i in range(10000):
    l,W,b = grad_descent(X,Y,W,b,learning_rate=0.1)
    acc.append(get_acc(x,test,W,b))
    loss.append(l)

In [60]:
print(X[0])
print(Y[0])
print(loss[0])

[ 3.    1.   22.    1.    0.    7.25  2.  ]
0
7.47757154366949


In [61]:
print(max(acc))

0.8995215311004785


In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
reg=LogisticRegression()

In [42]:
reg.fit(X,Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
score = reg.score(x,test)
print(score)

0.9401913875598086
